In [57]:
import os

In [58]:
%pwd

'D:\\Projects\\Chicken-Disease-Classification-Project\\Config'

In [59]:
os.chdir("../")

In [60]:
%pwd

'D:\\Projects\\Chicken-Disease-Classification-Project'

In [61]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [62]:
from ChickenDiseaseClassification.constants import *
from ChickenDiseaseClassification.utils.common import read_yaml, create_directories


In [67]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [68]:
import os
import urllib.request as request
import zipfile
from ChickenDiseaseClassification import logger
from ChickenDiseaseClassification.utils.common import get_size

In [69]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [70]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-11-17 20:00:07,669: INFO: common]: yaml file: config\config.yaml loaded successfully
[2023-11-17 20:00:07,671: INFO: common]: yaml file: params.yaml loaded successfully
[2023-11-17 20:00:07,672: INFO: common]: created directory at: artifacts
[2023-11-17 20:00:07,672: INFO: common]: created directory at: artifacts/data_ingestion
[2023-11-17 20:53:53,350: INFO: 4037744573]: artifacts/data_ingestion/ChickenDiseaseData.zip download! with following info: 
x-amz-id-2: 5pAttdAzHLVGZkeWtVsvXDZWNHuLIFfmoB2vGo82fx6sMPVxvrC9SLcLMBFnA6ciWk1CRbSOJug=
x-amz-request-id: FE6N6DZ2Q6K5J5W3
Date: Fri, 17 Nov 2023 20:00:08 GMT
Last-Modified: Fri, 17 Nov 2023 03:06:48 GMT
ETag: "030a45d256432a8fa46d45c8ea1ed799-17"
x-amz-server-side-encryption: AES256
Accept-Ranges: bytes
Content-Type: application/zip
Server: AmazonS3
Content-Length: 278791023
Connection: close


